<a href="https://colab.research.google.com/github/masunomatiko/lab/blob/master/%E3%82%B5%E3%83%BC%E3%83%90%E3%81%A7MySQL%E4%BD%BF%E3%81%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## やりたいこと
DBpediaのinternal linksのファイルが大きすぎてラップトップで扱えないので、**サーバでMySQLに入れてデータを抽出する**

### インストール
```
sudo apt update
sudo apt install mysql-server mysql-client
```
※サーバ係の人にsudo権限つけてもらうこと

データベース作るとかテーブル作るとかは普通のMySQLと変わらない
こんな感じで作った↓
```
mysql> show tables
    -> ;
+--------------------+
| Tables_in_mrep |
+--------------------+
| link                       |
+--------------------+
1 row in set (0.00 sec)
```

### .ttlファイルからロードする
このSQLで `page_links_wkd_uris_en.ttl` をlinkテーブルに入れる
```
mysql> LOAD DATA INFILE 'page_links_wkd_uris_en.ttl' INTO TABLE link
    -> FIELDS TERMINATED BY ' '
    -> LINES TERMINATED BY ' .'
    -> IGNORE 1 LINES
    -> ;
```
まず出たエラー
```
ERROR 1290 (HY000): The MySQL server is running with the --secure-file-priv option so it cannot execute this statement
```
このエラーでググって[見たページ](http://proengineer.internous.co.jp/content/columnfeature/6776#section200)を読んで `secure_file_priv` を空にしたらいいらしいと検討をつける

で、 `my.cnf` を探す
```
私の名前@サーバの名前:~/ディレクトリ$ mysql --help | grep my.cnf
                      order of preference, my.cnf, $MYSQL_TCP_PORT,
/etc/my.cnf /etc/mysql/my.cnf ~/.my.cnf
```
左から順によばれるのでいちばん最後の `~/.my.cnf` を編集すると良い

デフォルトでは `/etc/mysql/my.cnf` しかないのでこれに書けばいいのかなという感じだけど変わらなかった（[変わることもある](https://superuser.com/questions/1088512/how-to-disable-secure-file-priv-mysql-ubuntu)）ので `~/.my.cnf` を作成して編集
```
[mysqld]
secure_file_priv=""
```
MySQLを再起動するとちゃんと読み込まれてた（編集する前はValueにデフォルトのパスが書いてある）
```
mysql> SHOW VARIABLES LIKE "secure_file_priv";
+---------------------+-------+
| Variable_name    | Value |
+---------------------+-------+
| secure_file_priv  |           |
+---------------------+-------+
1 row in set (0.00 sec)
```


次に出たエラー
```
ERROR 13 (HY000): Can't get stat of '/var/lib/mysql/mrep/page_links_wkd_uris_en.ttl' (Errcode: 2 - No such file or directory)
```
LOAD DATA INFILEではファイルはサーバーホスト上（`/var/lib/mysql/` とか）にある必要があり、直接サーバによって読み取られる。

LOAD DATA **LOCAL** INFILEという文だとMySQLのクライアント側にファイルを読み込ませることができるので今回はそっちを使う。
```
mysql> LOAD DATA LOCAL INFILE 'page_links_wkd_uris_en.ttl' INTO TABLE link
    -> FIELDS TERMINATED BY ' '
    -> LINES TERMINATED BY ' .'
    -> IGNORE 1 LINES
    -> ;
```
結果（速い）
```
Query OK, 138411792 rows affected (17 min 40.84 sec)
Records: 138411792  Deleted: 0  Skipped: 0  Warnings: 0
```